In this notebook, we'll investigate the optimal properties and color sets by simulating a monopoly game 10000 times to estimate the expected value of the number of times a player lands on a property. We'll assume that games will terminate when only one player is remaining, or after 250 turns (as most games continue in perpetuitity). Players are set to a normal level of aggression.

In [1]:
import init
import pandas as pd

pd.set_option('display.max_columns', 500)

In [2]:
# Initialize frequency of every property
d = {n:0 for n in range(40)}
# Labeling of every property
name = {
    0:'Go',
    1:'Mediterranean',
    2:'Community Chance 1',
    3:'Baltic',
    4:'Income Tax',
    5:'Reading RR',
    6:'Oriental',
    7:'Chance 1',
    8:'Vermont',
    9:'Connecticut',
    10:'Jail',
    11:'St. Charles',
    12:'Electric Company',
    13:'States',
    14:'Virginia',
    15:'Pennsylvania RR',
    16:'St. James',
    17:'Community Chest 2',
    18:'Tennessee',
    19:'New York',
    20:'Free Parking',
    21:'Kentucky',
    22:'Chance 2',
    23:'Indiana',
    24:'Illinois',
    25:'B & O RR',
    26:'Atlantic',
    27:'Ventnor',
    28:'Water Works',
    29:'Marvin Gardens',
    30:'Go to Jail',
    31:'Pacific',
    32:'North Carolina',
    33:'Community Chest 3',
    34:'Pennsylvania',
    35:'Short Line RR',
    36:'Chance 3',
    37:'Park Place',
    38:'Luxury Tax',
    39:"Boardwalk"
}

# Simulate game 10000 times:
for _ in range(10000):
    results = init.main(250, 0, ['Default', 'Default', 'Default', 'Default'])
    for key in d:
        d[key] += results[key]

df = pd.DataFrame(d, index = [0])
df.columns = [name[col] for col in df.columns]
view = df.T
view

,0
Go,129078
Mediterranean,59841
Community Chance 1,53867
Baltic,62549
Income Tax,69151
Reading RR,86872
Oriental,69428
Chance 1,25267
Vermont,71184
Connecticut,70196


From here, let's focus our view on normal properties (properties where houses/hotels can be built). By calculating the expected number of times an opposing player lands on a certain property throughout a game, we can determine the expected return (given in basis points) every property with every possible number of houses/hotels.

In [3]:
# Filter to only normal properties
normal = {
    1:'Mediterranean',
    3:'Baltic',
    6:'Oriental',
    8:'Vermont',
    9:'Connecticut',
    11:'St. Charles',
    13:'States',
    14:'Virginia',
    16:'St. James',
    18:'Tennessee',
    19:'New York',
    21:'Kentucky',
    23:'Indiana',
    24:'Illinois',
    26:'Atlantic',
    27:'Ventnor',
    29:'Marvin Gardens',
    31:'Pacific',
    32:'North Carolina',
    34:'Pennsylvania',
    37:'Park Place',
    39:"Boardwalk"
}

normal_properties = df.copy()

for col in df.columns:
    if col not in normal.values():
        del normal_properties[col]

In [4]:
# Method that creates new column with expected return of investment
def create_row(val, df):
    tmp = []

    for col in df.columns:
        for k, v in name.items():
            if v == col:
                g = init.Game(0, ['Default', 'Default'])
                p = g.Go
                while p.number != k:
                    p = p.next
                if val == 'rent':
                    tmp.append(int(100*(0.75*(df.loc[1][col]-1)*p.rent)/max(p.cost,1)))
                elif val == 'one_house':
                    tmp.append(int(100*(0.75*(df.loc[1][col])*p.one_house)/max(p.cost+p.cost_of_house,1)))
                elif val == 'two_houses':
                    tmp.append(int(100*(0.75*(df.loc[1][col])*p.two_houses)/max(p.cost+2*p.cost_of_house,1)))
                elif val == 'three_houses':
                    tmp.append(int(100*(0.75*(df.loc[1][col])*p.three_houses)/max(p.cost+3*p.cost_of_house,1)))
                elif val == 'four_houses':
                    tmp.append(int(100*(0.75*(df.loc[1][col])*p.four_houses)/max(p.cost+4*p.cost_of_house,1)))
                elif val == 'hotel':
                    tmp.append(int(100*(0.75*(df.loc[1][col])*p.hotel)/max(p.cost+5*p.cost_of_house,1)))
                break

    return tmp

In [5]:
# Estimate return on investment
normal_properties.loc[1] = [normal_properties.loc[0][col]/10000 for col in normal_properties.columns]
cols = ['rent', 'one_house', 'two_houses', 'three_houses', 'four_houses', 'hotel']

for i, val in enumerate(cols):
    tmp = create_row(val, normal_properties)
    normal_properties.loc[i+2] = tmp.copy()

normal_properties

,Mediterranean,Baltic,Oriental,Vermont,Connecticut,St. Charles,States,Virginia,St. James,Tennessee,New York,Kentucky,Indiana,Illinois,Atlantic,Ventnor,Marvin Gardens,Pacific,North Carolina,Pennsylvania,Park Place,Boardwalk
0,59841.0000,62549.0000,69428.0000,71184.0000,70196.0000,83076.0000,69750.000,72740.000,83043.0000,86379.0000,91477.0000,82364.0000,79308.0000,95057.0000,77732.0000,77632.0000,74009.0000,76354.0000,74126.0000,70946.0000,61252.0000,75402.0000
1,5.9841,6.2549,6.9428,7.1184,7.0196,8.3076,6.975,7.274,8.3043,8.6379,9.1477,8.2364,7.9308,9.5057,7.7732,7.7632,7.4009,7.6354,7.4126,7.0946,6.1252,7.5402
2,12.0000,26.0000,26.0000,27.0000,30.0000,39.0000,32.000,35.000,42.0000,44.0000,48.0000,44.0000,42.0000,53.0000,42.0000,42.0000,41.0000,43.0000,41.0000,39.0000,38.0000,61.0000
3,40.0000,85.0000,104.0000,106.0000,123.0000,129.0000,108.000,125.000,155.0000,161.0000,182.0000,150.0000,144.0000,182.0000,156.0000,156.0000,154.0000,148.0000,144.0000,153.0000,146.0000,188.0000
4,84.0000,175.0000,234.0000,240.0000,239.0000,274.0000,230.000,272.000,327.0000,340.0000,377.0000,296.0000,285.0000,396.0000,343.0000,343.0000,344.0000,319.0000,309.0000,332.0000,306.0000,424.0000
5,192.0000,402.0000,562.0000,576.0000,584.0000,637.0000,535.000,592.000,713.0000,742.0000,823.0000,645.0000,621.0000,774.0000,656.0000,656.0000,646.0000,572.0000,555.0000,578.0000,531.0000,791.0000
6,276.0000,577.0000,694.0000,711.0000,740.0000,721.0000,605.000,681.000,805.0000,837.0000,914.0000,659.0000,634.0000,785.0000,660.0000,660.0000,646.0000,572.0000,555.0000,570.0000,519.0000,801.0000
7,361.0000,680.0000,818.0000,838.0000,853.0000,730.0000,613.000,743.000,870.0000,905.0000,980.0000,668.0000,643.0000,792.0000,663.0000,662.0000,646.0000,561.0000,545.0000,564.0000,510.0000,807.0000


In [6]:
# Standardize outputs and sort by property rankings
normal_properties_standardized = normal_properties.copy().T
normal_properties_standardized.columns = ['Property', 'Expected Landings', 'ROI Rent', 'ROI 1 House',
                                          'ROI 2 Houses', 'ROI 3 Houses', 'ROI 4 Houses', 'ROI Hotel']
for col in normal_properties_standardized.columns:
    normal_properties_standardized[col] = (normal_properties_standardized[col] -
                           normal_properties_standardized[col].mean()) / normal_properties_standardized[col].std()


normal_properties_standardized['Mean Z-Score'] = normal_properties_standardized[['ROI Rent', 'ROI 1 House',
                                          'ROI 2 Houses', 'ROI 3 Houses', 'ROI 4 Houses', 'ROI Hotel']].mean(axis=1)
normal_properties_standardized = normal_properties_standardized.sort_values(by='Mean Z-Score', ascending=False)
normal_properties_standardized


,Property,Expected Landings,ROI Rent,ROI 1 House,ROI 2 Houses,ROI 3 Houses,ROI 4 Houses,ROI Hotel,Mean Z-Score
New York,1.757758,1.757758,0.923240,1.279852,1.085427,1.595074,1.875556,1.877777,1.439488
Boardwalk,-0.025213,-0.025213,2.186621,1.454920,1.707218,1.357316,1.025641,0.707702,1.406570
Illinois,2.154836,2.154836,1.409156,1.279852,1.336790,1.231007,0.905299,0.606250,1.128059
Tennessee,1.192309,1.192309,0.534507,0.667115,0.595933,0.993248,1.296410,1.370519,0.909622
St. James,0.822294,0.822294,0.340141,0.492047,0.423948,0.777780,1.055727,1.133799,0.703907
Atlantic,0.233221,0.233221,0.340141,0.521225,0.635621,0.354273,-0.034872,-0.266234,0.258359
Ventnor,0.222129,0.222129,0.340141,0.521225,0.635621,0.354273,-0.034872,-0.272997,0.257232
Marvin Gardens,-0.179719,-0.179719,0.242958,0.462869,0.648851,0.279974,-0.140171,-0.381212,0.185545
Kentucky,0.746982,0.746982,0.534507,0.346157,0.013831,0.272544,-0.042393,-0.232416,0.148705
St. Charles,0.825954,0.825954,0.048592,-0.266581,-0.277220,0.213104,0.423932,0.186917,0.054791


Interestingly, Boardwalk is our second most cost-effective property, aided by its strong expected ROI in the early-game when houses have yet to be built. If considering late game scenarios where we can expect houses/hotels to be built often, the Orange Set (New York, Tennesse, and St. James) reign surpreme. This is likely explained by their proximity to being one expected roll after a player exits a jail, another popular square. Illinois is also another great property, aided by being located between Jail and Go to Jail, and the ability for players to reach it by Chance cards.

Perhaps more importantly, our data shows that no property generated a positive ROI without houses/hotels being built. For our worst property (Mediterranean), three houses need to be built before profit is expected positive. This indicates that optimal strategy likely prioritizes building houses/hotels as fast as possible.

This data can be viewed in csv form under 'data/normal_properties_standardized'

In [7]:
normal_properties_standardized.to_csv('data/normal_properties_standardized.csv')

Now let's consider the railroads. Using a similar process, we can generate data for the expected ROI of railroad purchases.

In [8]:
rr = {
    5:'Reading RR',
    15:'Pennsylvania RR',
    25:'B & O RR',
    35:'Short Line RR',
}

railroads = df.copy()

for col in df.columns:
    if col not in rr.values():
        del railroads[col]
railroads.loc[1] = [railroads.loc[0][col]/10000 for col in railroads.columns]
railroads.loc[2] = [int(100*(0.75*(railroads.loc[1][col]-1)*50)/200) for col in railroads.columns]
railroads.loc[3] = [int(100*(0.75*(railroads.loc[1][col]-1)*50*2)/400) for col in railroads.columns]
railroads.loc[4] = [int(100*(0.75*(railroads.loc[1][col]-1)*50*3)/600) for col in railroads.columns]
railroads.loc[5] = [int(100*(0.75*(railroads.loc[1][col]-1)*50*4)/800) for col in railroads.columns]

railroads = railroads.T
railroads.columns = ['Property', 'Expected Landings', 'One RR', 'Two RR', 'Three RR', 'Four RR']

railroads

,Property,Expected Landings,One RR,Two RR,Three RR,Four RR
Reading RR,86872.0,8.6872,144.0,144.0,144.0,144.0
Pennsylvania RR,83008.0,8.3008,136.0,136.0,136.0,136.0
B & O RR,84029.0,8.4029,138.0,138.0,138.0,138.0
Short Line RR,69010.0,6.9010,110.0,110.0,110.0,110.0


With only a few, similar properties, standardizing these values won't reveal many new insights. Unlike the normal properties, Railroads immediately provide a positive ROI. The worst performing railroad (Short Line) expects a ROI of 10% while the best performing normal property (Boardwalk) without any houses expects a ROI of -39%. Railroads seem to serve a strong early game strategic purpose, as one of the few positively performing investments. However, in the long run, railroad's expected ROI pales in comparison to normal properties.

One key item to notice is that a railroad's expected ROI does not change with additional railroads. To find the expected ROI of multiple railroads, you would simply average the expected ROI of each of them. Reading expectedly outperforms the rest due to the ability for players to travel to reading through Chance cards. B & O and Pennsylvania are next, due to being placed between jail and Go to Jail.

This data can be viewed in csv form under 'data/railroads.csv'

In [9]:
railroads.to_csv('data/railroads.csv')

Finally, we can examine Utility properties. Similar to above, we can calculate the expected ROI of these properties.

In [10]:
utiity = {
    12:'Electric Company',
    28:'Water Works'
}

utilities = df.copy()

for col in df.columns:
    if col not in utiity.values():
        del utilities[col]
utilities.loc[1] = [utilities.loc[0][col]/10000 for col in utilities.columns]
utilities.loc[2] = [int(100*(0.75*(utilities.loc[1][col]-1)*28)/150) for col in utilities.columns]
utilities.loc[3] = [int(100*(0.75*(utilities.loc[1][col]-1)*70)/300) for col in utilities.columns]


utilities = utilities.T
utilities.columns = ['Property', 'Expected Landings', 'One Utility', 'Two Utility']

utilities

,Property,Expected Landings,One Utility,Two Utility
Electric Company,78191.0,7.8191,95.0,119.0
Water Works,82174.0,8.2174,101.0,126.0


Unlike the railroads, the utilities don't provide immediate positive expected return while also having very limited upside. Without buying both utilies, a player is at an expected loss. Even if a player posses both Utilites, they are still expected to earn roughly 23% throughout a game. Compared to other options, it's clear that Utilites are by far the weakest properties to own.

Finally, as a general caveat, it's important to understand that these calculations are made off of expected number of landings. Thus, the actual ROI of normal properites with many houses/hotels will be much lower than calculated, as it would be highly unlikely all of the expected landings occur with a large number of houses/hotels already built.